# Data Transfer - Goes16

This notebook helps outline a transfer process for managing a data transfer from RCC's Minio Server to the OSDC Griffin Ceph storage.   It relies on using AWS cli as a python subprocess.   This is necessary (instead of say - boto) because OSDC s3 utilizes a version of Ceph that necessitates awsv2 signitures.   

We rely on awscli < 1.09 to manage this issue and connect to both resources. 

`pip install "awscli>=1.0.0,<=1.09.0"`

Otherwise the goal her is to create scripts to transfer event data for Hurricanes Irma, Maria, and Jose (all >= category 4) to OSDC Griffin to make these easier for the community to use/study.

The data is at: https://griffin-objstore.opensciencedatacloud.org/noaa-goes16-hurricane-archive-2017/

For more information visit:  http://edc.occ-data.org/

In [4]:
import subprocess, os, shutil

# Create Bucket

## Delete If Doesn't Exist

In [20]:
deletebucket = 'aws s3 rb s3://noaa-goes16-hurricane-archive-2017/ --endpoint-url https://griffin-objstore.opensciencedatacloud.org --force'
deleteB=subprocess.Popen(deletebucket, shell=True, stdout = subprocess.PIPE)

##  Make Bucket

In [21]:
makebucket = 'aws s3 mb s3://noaa-goes16-hurricane-archive-2017/ --endpoint-url https://griffin-objstore.opensciencedatacloud.org'
makeB=subprocess.Popen(makebucket, shell=True, stdout = subprocess.PIPE)

# Data Transfer

In [46]:
sensor = 'ABI-L1b-RadC'
hurricane = 'irma'
day = '355'
localdir = ' ~/Hurricane/.'

In [47]:
rccloc = 's3://noaa-goes16/' + sensor + '/2017/' + day + '/'

cmd = 'aws s3 cp ' + rccloc + localdir + ' --no-sign-request --endpoint-url https://osdc.rcc.uchicago.edu --no-verify-ssl --recursive'
cmd

'aws s3 cp s3://noaa-goes16/ABI-L1b-RadC/2017/355/ ~/Hurricane/. --no-sign-request --endpoint-url https://osdc.rcc.uchicago.edu --no-verify-ssl --recursive'

In [25]:
push=subprocess.Popen(cmd, shell=True, stdout = subprocess.PIPE)

In [48]:
bucketloc = ' s3://noaa-goes16-hurricane-archive-2017/' + hurricane + '/' + sensor + '/' + day + '/'
pushcmd = 'aws s3 cp ' + localdir + bucketloc + ' --endpoint-url https://griffin-objstore.opensciencedatacloud.org --acl public-read --recursive'
pushcmd

'aws s3 cp  ~/Hurricane/. s3://noaa-goes16-hurricane-archive-2017/irma/ABI-L1b-RadC/355/ --endpoint-url https://griffin-objstore.opensciencedatacloud.org --acl public-read --recursive'

In [27]:
push=subprocess.Popen(pushcmd, shell=True, stdout = subprocess.PIPE)

In [19]:
## delete folder contents
def cleanhouse(folder):
    # https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder-in-python
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)

In [39]:
folder = '/home/wells_walt/Hurricane'
cleanhouse(folder)

# Make Bucket Public

In [28]:
makepublic = 'aws s3api put-bucket-acl --endpoint-url https://griffin-objstore.opensciencedatacloud.org --bucket noaa-goes16-hurricane-archive-2017 --acl public-read'
makePub=subprocess.Popen(makepublic, shell=True, stdout = subprocess.PIPE)